#### Thursday, April 4, 2024

https://microsoft.github.io/autogen/docs/tutorial/introduction


In [4]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="TheBloke/NexusRaven-V2-13B-GGUF/nexusraven-v2-13b.Q8_0.gguf",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Always answer in rhymes.', role='assistant', function_call=None, tool_calls=None)


# Introduction to AutoGen

Welcome! AutoGen is an open-source framework that leverages multiple _agents_ to enable complex workflows. This tutorial introduces basic concepts and building blocks of AutoGen.

## Why AutoGen?

> _The whole is greater than the sum of its parts._<br/>
> -**Aristotle**

While there are many definitions of agents, in AutoGen, an agent is an entity that can send messages, receive messages and generate a reply using models, tools, human inputs or a mixture of them.
This abstraction not only allows agents to model real-world and abstract entities, such as people and algorithms, but it also simplifies implementation of complex workflows as collaboration among agents.

Further, AutoGen is extensible and composable: you can extend a simple agent with customizable components and create workflows that can combine these agents and power a more sophisticated agent, resulting in implementations that are modular and easy to maintain.

Most importantly, AutoGen is developed by a vibrant community of researchers
and engineers. It incorporates the latest research in multi-agent systems
and has been used in many real-world applications, including agent platform,
advertising, AI employees, blog/article writing, blockchain, calculate burned areas by wildfires,
customer support, cybersecurity, data analytics, debate, education, finance, gaming, legal consultation,
research, robotics, sales/marketing, social simulation, software engineering,
software security, supply chain, t-shirt design, training data generation, Youtube service...

## Installation

The simplest way to install AutoGen is from pip: `pip install pyautogen`. Find more options in [Installation](/docs/installation/).

## Agents

In AutoGen, an agent is an entity that can send and receive messages to and from
other agents in its environment. An agent can be powered by models (such as a large language model
like GPT-4), code executors (such as an IPython kernel), human, or a combination of these
and other pluggable and customizable components.

```{=mdx}
![ConversableAgent](./assets/conversable-agent.jpg)
```

An example of such agents is the built-in `ConversableAgent` which supports the following components:

1. A list of LLMs
2. A code executor
3. A function and tool executor
4. A component for keeping human-in-the-loop

You can switch each component on or off and customize it to suit the need of 
your application. For advanced users, you can add additional components to the agent
by using [`registered_reply`](../reference/agentchat/conversable_agent/#register_reply). 

LLMs, for example, enable agents to converse in natural languages and transform between structured and unstructured text. 
The following example shows a `ConversableAgent` with a GPT-4 LLM switched on and other
components switched off:

In [1]:
# Yeah we are NOT goind to use OpenAI ...
config_list=[
    {
        "model": "LMStudio Local Model",
        "base_url": "http://localhost:1234/v1",
        "api_type": "open_ai",
        "api_key": "NULL", # required, you don´t need to change this
        }
]

llm_config={
    "config_list": config_list,
}

In [2]:
import os
from autogen import ConversableAgent

agent = ConversableAgent(
    "chatbot",
    # llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ.get("OPENAI_API_KEY")}]},
    llm_config = llm_config, 
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

The `llm_config` argument contains a list of configurations for the LLMs.
See [LLM Configuration](/docs/topics/llm_configuration) for more details.

You can ask this agent to generate a response to a question using the `generate_reply` method:

In [3]:
reply = agent.generate_reply(messages=[{"content": "Tell me a joke.", "role": "user"}])
print(reply)

No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


Sure! Here's a joke: Why don't lobsters share? Because they're shellfish!


## Roles and Conversations

In AutoGen, you can assign roles to agents and have them participate in conversations or chat with each other. A conversation is a sequence of messages exchanged between agents. You can then use these conversations to make progress on a task. For example, in the example below, we assign different roles to two agents by setting their
`system_message`.

In [5]:
cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    # llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    llm_config = llm_config ,
    human_input_mode="NEVER",  # Never ask for human input.
)

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    # llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    llm_config = llm_config ,
    human_input_mode="NEVER",  # Never ask for human input.
)

Now that we have two comedian agents, we can ask them to start a comedy show.
This can be done using the `initiate_chat` method.
We set the `max_turns` to 2 to keep the conversation short.

In [6]:
result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.", max_turns=2)

No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


cathy (to joe):

Okay, I'll give it a try: "Why did the scarecrow win an award? Because he was outstanding in his field!"

This is a play on words, as the word "outstanding" can also refer to being excellent or exceptional, which is why the scarecrow won the award. It's a clever pun that will make you laugh!

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


joe (to cathy):

Ha ha, that's a great joke!

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


cathy (to joe):

Thank you! I'm glad you enjoyed it. Do you want to hear another one?

--------------------------------------------------------------------------------


The comedians are bouncing off each other!

## Summary

In this chapter, we introduced the concept of agents, roles and conversations in AutoGen.
For simplicity, we only used LLMs and created fully autonomous agents (`human_input_mode` was set to `NEVER`). 
In the next chapter, 
we will show how you can control when to _terminate_ a conversation between autonomous agents.